# LGBM experiments

In [23]:
import shap
import pandas as pd
import numpy as np
import time
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb

from sklearn.metrics import (
    r2_score, max_error, mean_absolute_error, mean_squared_error,
    mean_absolute_percentage_error, make_scorer
)
from sklearn.model_selection import (
    TimeSeriesSplit, train_test_split, cross_val_score)

from glob import glob
import os
import optuna
import joblib
from tqdm import tqdm
import json

import warnings
warnings.filterwarnings('ignore')

from datetime import datetime

In [2]:
import psutil
    
ram_gb = psutil.virtual_memory().total / 2**30 # total physical memory in bytes
print(f"RAM: {ram_gb:.2f} GB")

RAM: 251.54 GB


In [3]:
# preprocess utils

# compress dtypes
dtypes_compressed = {
    'datetime': np.object_,    
    'hour': np.int8,
    'month': np.int8,
    'dayofweek': np.int8,
    'number_of_lanes': np.float32,
    
    'speed_limit_kph': np.float32,    
    
    'lon': np.float64,
    'lat': np.float64,
    'elevation': np.int16,    
    'agg_speed': np.float64,  
    
    'pix_business_4x4_200': np.int16,
    'pix_residential_4x4_200': np.int16,
    'pix_industrial_4x4_200': np.int16,
    'pix_institutional_4x4_200': np.int16,
    'pix_business_4x4_50': np.int16,
    'pix_residential_4x4_50': np.int16,
    'pix_industrial_4x4_50': np.int16,
    'pix_institutional_4x4_50': np.int16,
    'pix_business_4x4_100': np.int16,
    'pix_residential_4x4_100': np.int16,
    'pix_industrial_4x4_100': np.int16,
    'pix_institutional_4x4_100': np.int16,
    'pix_business_4x4_150': np.int16,
    'pix_residential_4x4_150': np.int16,
    'pix_industrial_4x4_150': np.int16,
    'pix_institutional_4x4_150': np.int16,
    'pix_business_4x4_250': np.int16,
    'pix_residential_4x4_250': np.int16,
    'pix_industrial_4x4_250': np.int16,
    'pix_institutional_4x4_250': np.int16,
    'pix_business_4x4_500': np.int16,
    'pix_residential_4x4_500': np.int16,
    'pix_industrial_4x4_500': np.int16,
    'pix_institutional_4x4_500': np.int16,
    'pix_business_4x4_1000': np.int16,
    'pix_residential_4x4_1000': np.int16,
    'pix_industrial_4x4_1000': np.int16,
}

def correct_service_rd_kphlimit(temp_df):
    temp_df.loc[temp_df['speed_limit_kph'] == 36.7, 'speed_limit_kph'] = 20
    return temp_df

def impute_lanes(df):
    df['number_of_lanes'] = df['number_of_lanes'].replace(np.nan, value=2)
    return df

def filter_brgy(df):
    df['barangay'] = df['barangay'].fillna('Out-of-town')
    
    # not in cauayan boundary
    df = df.loc[df['barangay'] != 'Out-of-town'] 
    return df

def filter_num_periods(df, thresh):
    df = df.loc[df['num_periods'] >= thresh]
    return df

def filter_vehicles(df, remove_list=None):
    df = df.loc[~df.vehicle_id.isin(remove_list)]
    return df

def prepare_df(df):
    df = df.astype(dtype=dtypes_compressed)
    
    # vehicles with erratic sending of data; stopped sending data before collection date
    remove_list = [
        123200872653, 123200872678, 
        123200872713, 123200872727, 123200872819,
    ] 
    df = filter_brgy(df)
    df = filter_vehicles(df, remove_list=remove_list)
    df = filter_num_periods(df, thresh=12)
    df = impute_lanes(df)
    df = correct_service_rd_kphlimit(df)
    df = df.drop_duplicates()
    return df

In [4]:
# prepare rmse scorer
def rmse(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    return rmse
rmse_scorer = make_scorer(rmse) # rmse scoring metric for cross_val

metrics_dict = {
    "r2" : r2_score,
    "mean_absolute_error" : mean_absolute_error, 
    "mean_absolute_percentage_error" : mean_absolute_percentage_error,     
    "rmse" : rmse,
    "mean_squared_error" : mean_squared_error,    
    "max_error": max_error,
}

## Load data

In [38]:
# experiment_type = "all_features" 
experiment_type = "landuse_and_time" 
# experiment_type = "landuse_only" 

buffers = 'all'
# buffers = [50, 100, 250, 200, 250, 500]

In [40]:
src_date = "2023-06-17"     
src_dir = f"../datasets/processed/post-review-tt-splits/{src_date}"
tune = pd.read_csv(os.path.join(src_dir, "tune.csv"))
tune = prepare_df(tune)

In [41]:
# target
target = 'agg_speed'

# features
time_cols = ['hour', 'dayofweek']

road_cols = [
    'number_of_lanes', 'speed_limit_kph', 'elevation',
]

landuse_cols = []
if buffers=="all":
    buffer_sizes = [50, 100, 150, 200, 250, 500, 1000]
else:
    buffer_sizes = buffers
    
for size in buffer_sizes:
    landuse_cols.extend(
        [
            f'pix_residential_4x4_{size}',
            f'pix_institutional_4x4_{size}',
            f'pix_industrial_4x4_{size}',
            f'pix_business_4x4_{size}',
        ]
    )
cat_cols = time_cols

if experiment_type == "all_features":
    features = time_cols + road_cols + landuse_cols
    cat_cols_index = [
        index for (index, col) in enumerate(features) if col in cat_cols]

elif experiment_type == "landuse_and_time":
    features = time_cols + landuse_cols
    cat_cols_index = [
        index for (index, col) in enumerate(features) if col in cat_cols]

else:
    features = landuse_cols
    cat_cols_index = None

In [42]:
features

['hour',
 'dayofweek',
 'pix_residential_4x4_50',
 'pix_institutional_4x4_50',
 'pix_industrial_4x4_50',
 'pix_business_4x4_50',
 'pix_residential_4x4_100',
 'pix_institutional_4x4_100',
 'pix_industrial_4x4_100',
 'pix_business_4x4_100',
 'pix_residential_4x4_150',
 'pix_institutional_4x4_150',
 'pix_industrial_4x4_150',
 'pix_business_4x4_150',
 'pix_residential_4x4_200',
 'pix_institutional_4x4_200',
 'pix_industrial_4x4_200',
 'pix_business_4x4_200',
 'pix_residential_4x4_250',
 'pix_institutional_4x4_250',
 'pix_industrial_4x4_250',
 'pix_business_4x4_250',
 'pix_residential_4x4_500',
 'pix_institutional_4x4_500',
 'pix_industrial_4x4_500',
 'pix_business_4x4_500',
 'pix_residential_4x4_1000',
 'pix_institutional_4x4_1000',
 'pix_industrial_4x4_1000',
 'pix_business_4x4_1000']

## Tuning

In [43]:
X, y = tune[features], tune[target]

# set up cv
tscv = TimeSeriesSplit(n_splits=3)

In [44]:
def objective(trial):
    """Objective function for optuna tuning. Uses LGBM sklearn API."""
    lgb_params = {
        'tree_learner': 'data',
        'seed':11,
        'verbose': -1,
#         'boosting_type': trial.suggest_categorical(
#             'boosting_type', ['goss', 'dart']),
        'boosting_type': 'goss',
        
        'objective': trial.suggest_categorical(
            'objective', ['regression', 'tweedie', 'regression_l1', 'mape']),
        
        'num_leaves': trial.suggest_int('num_leaves', 2, 512, log=True),
        'min_data_in_leaf': trial.suggest_int(
            'min_data_in_leaf', 100, 10000, log=True),
        
        'max_bin': trial.suggest_int(
            'max_bin', 100, 300),
        
        'n_estimators': trial.suggest_int("n_estimators", 100, 1000),
        'learning_rate' : trial.suggest_float("learning_rate", 0.001, 1, log=True),
    }
    if lgb_params['boosting_type']=='goss':
        top_rate =  trial.suggest_float("top_rate", 0, 1.0)
        other_rate = trial.suggest_float(
            "other_rate", 0, (1.0 - top_rate)) 

        # corrects the top_rate/other_rate values to only sum to 1
        if (top_rate + other_rate) > 1.0:
            lgb_params['other_rate'] = other_rate / (top_rate + other_rate)
            lgb_params['top_rate'] = top_rate / (top_rate + other_rate)
        

    model = lgb.LGBMRegressor(**lgb_params)
    scores = cross_val_score(
        estimator=model, X=X, y=y, scoring=rmse_scorer, cv=tscv, n_jobs=-1, 
        fit_params={'categorical_feature': cat_cols_index},
        error_score='raise')
    
    return np.mean(scores)

In [45]:
study_name = f'{experiment_type}'
if buffers=='all':
    buffers = buffers
else:
    buffers = "-".join([str(buffers[0]), str(buffers[-1])])
    
save_dir = f'../experiments/post-review/{buffers}/{src_date}'
os.makedirs(save_dir, exist_ok=True)
    
study = optuna.create_study(study_name=study_name,
                            direction='minimize',
                            storage=f'sqlite:///{save_dir}/{study_name}.db',
                            load_if_exists=True)

[I 2023-06-19 02:24:41,211] Using an existing study with name 'landuse_and_time' instead of creating a new one.


In [46]:
study.optimize(objective, n_trials=50) 

/home/mdorosan/.local/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
[I 2023-06-19 02:25:08,997] Trial 20 finished with value: 6.594664754387733 and parameters: {'learning_rate': 0.11275982502389782, 'max_bin': 253, 'min_data_in_leaf': 183, 'n_estimators': 220, 'num_leaves': 218, 'objective': 'regression', 'other_rate': 0.13272869611166377, 'top_rate': 0.44394980131002887}. Best is trial 14 with value: 6.37340348011161.
/home/mdorosan/.local/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning: Using categ

[I 2023-06-19 02:29:31,933] Trial 29 finished with value: 6.439959517272743 and parameters: {'learning_rate': 0.005041810411515611, 'max_bin': 115, 'min_data_in_leaf': 290, 'n_estimators': 482, 'num_leaves': 148, 'objective': 'regression_l1', 'other_rate': 0.02224061790815196, 'top_rate': 0.9326118084755468}. Best is trial 24 with value: 6.356694463710485.
/home/mdorosan/.local/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
[I 2023-06-19 02:29:47,541] Trial 30 finished with value: 6.4565064715072635 and parameters: {'

/home/mdorosan/.local/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
[I 2023-06-19 02:33:04,028] Trial 39 finished with value: 6.413168410963756 and parameters: {'learning_rate': 0.06827904569244506, 'max_bin': 134, 'min_data_in_leaf': 188, 'n_estimators': 508, 'num_leaves': 45, 'objective': 'regression_l1', 'other_rate': 0.041360081657837736, 'top_rate': 0.8984722491330741}. Best is trial 32 with value: 6.349258388522691.
/home/mdorosan/.local/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning: Using ca

[I 2023-06-19 02:37:49,234] Trial 48 finished with value: 7.191470787020843 and parameters: {'learning_rate': 0.0010258006697377775, 'max_bin': 162, 'min_data_in_leaf': 252, 'n_estimators': 756, 'num_leaves': 203, 'objective': 'regression_l1', 'other_rate': 0.06950555622194168, 'top_rate': 0.7156165040773012}. Best is trial 32 with value: 6.349258388522691.
/home/mdorosan/.local/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
[I 2023-06-19 02:38:59,585] Trial 49 finished with value: 6.375568458330498 and parameters: {'

In [47]:
!ls ../experiments/post-review/

1000-1000  50-100  50-150  50-200  50-250  50-50  50-500  all


## Best hyperparams TSCV on train

In [49]:
params = study.best_params
score = study.best_value

print("Best params: ", params)
print("Best score: ", score)

fixed_params = {
    'tree_learner': 'data',
    'seed':11,
    'verbose': -1,
    'boosting_type': 'goss',
}

params.update(fixed_params)

Best params:  {'learning_rate': 0.021269226886571485, 'max_bin': 156, 'min_data_in_leaf': 101, 'n_estimators': 316, 'num_leaves': 72, 'objective': 'regression', 'other_rate': 0.04011983680490028, 'top_rate': 0.8519781632001752}
Best score:  6.349258388522691


In [50]:
src_dir = f"../datasets/processed/post-review-tt-splits/{src_date}"
train = pd.read_csv(os.path.join(src_dir, "train.csv"))
train = prepare_df(train)

test = pd.read_csv(os.path.join(src_dir, "test.csv"))
test = prepare_df(test)

[LightGBM] [Warning] min_data_in_leaf is set=148, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=148
[LightGBM] [Warning] min_data_in_leaf is set=174, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=174
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=148, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=148
[LightGBM] [Warning] min_data_in_leaf is set=174, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=174
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=148, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=148
[LightGBM] [Warning] min_data_in_leaf is set=174, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=174
[LightGB

In [51]:
# set up cv
tscv = TimeSeriesSplit(n_splits=5)
results = {}

for i, (train_index, test_index) in enumerate(tscv.split(train)):
    print(f"Current: Fold {i}")
    fold_train = train.iloc[train_index]
    fold_test = train.iloc[test_index]
    
    X_train, y_train = fold_train[features], fold_train[target]
    X_test, y_test = fold_test[features], fold_test[target]    
    
    model = lgb.LGBMRegressor(**params)
    model.fit(X_train, y_train, categorical_feature=cat_cols_index)

    preds = model.predict(X_test)
    true = y_test.values
    
    # log per fold
    results[i] = {
        "y_true": list(true),
        "y_pred": list(preds),
    }
    
    for metric, func in metrics_dict.items():
        results[i][metric] = func(true, preds)

Current: Fold 0
[LightGBM] [Warning] min_data_in_leaf is set=101, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=101
Current: Fold 1
[LightGBM] [Warning] min_data_in_leaf is set=101, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=101
Current: Fold 2
[LightGBM] [Warning] min_data_in_leaf is set=101, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=101
Current: Fold 3
[LightGBM] [Warning] min_data_in_leaf is set=101, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=101
[LightGBM] [Warning] min_data_in_leaf is set=145, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=145
[LightGBM] [Warning] min_data_in_leaf is set=249, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=249
[LightGBM] [Warning] min_data_in_leaf is set=130, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=130
Current: Fold 4
[LightGBM] [Warning] min_data_in_leaf is set=101, mi

In [52]:
tscv_results_dir = f"../datasets/results/post-review/tscv/{buffers}/{src_date}"
os.makedirs(tscv_results_dir, exist_ok=True)

fname = f"{experiment_type}.json"
with open(os.path.join(tscv_results_dir, fname), 'w') as fp:
    json.dump(results, fp)

[LightGBM] [Warning] min_data_in_leaf is set=145, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=145
[LightGBM] [Warning] min_data_in_leaf is set=249, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=249
[LightGBM] [Warning] min_data_in_leaf is set=130, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=130


In [102]:
!ls ../datasets/results/post-review/tscv/

1000-1000  150-150  250-250  50-100  50-200  50-50   all
100-100    200-200  500-500  50-150  50-250  50-500  baseline


## Evaluation on test

In [34]:
params

{'learning_rate': 0.0040562434730555055,
 'max_bin': 160,
 'min_data_in_leaf': 207,
 'n_estimators': 847,
 'num_leaves': 73,
 'objective': 'regression',
 'other_rate': 0.015101442936573879,
 'top_rate': 0.8715306418684673,
 'tree_learner': 'data',
 'seed': 11,
 'verbose': -1,
 'boosting_type': 'goss'}

In [35]:
X_train, y_train = train[features], train[target]
X_test, y_test = test[features], test[target]    

# retrain on full training set
model = lgb.LGBMRegressor(**params)
model.fit(X_train, y_train, categorical_feature=cat_cols_index)

# evaluate on holdout set
preds = model.predict(X_test)
true = y_test.values

# log final results
final_results = {
    "y_true": list(true),
    "y_pred": list(preds),
}

for metric, func in metrics_dict.items():
    final_results[metric] = func(true, preds)

[LightGBM] [Warning] min_data_in_leaf is set=207, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=207


In [36]:
final_results_dir = f"../datasets/results/post-review/final_results/{buffers}/{src_date}"
os.makedirs(final_results_dir, exist_ok=True)

fname = f"{experiment_type}.json"
with open(os.path.join(final_results_dir, fname), 'w') as fp:
    json.dump(final_results, fp)

In [37]:
!ls ../datasets/results/post-review/final_results/

50-100	50-150	50-200	50-250	50-50  50-500  all  baseline


## Automate runs

In [93]:
from itertools import product

buffer_ablation = [
    'all',
    [50, 100, 150, 200, 250, 500],
    [50, 100, 150, 200, 250],
    [50, 100, 150, 200],
    [50, 100, 150],
    [50, 100],
    [1000],
    [500],
    [250],
    [200],
    [150],
    [100],
    [50], 
]


etypes = [
    "all_features",
    "landuse_and_time", # queue this with the ablation
    "landuse_only",
]

exp_combis = product(etypes, buffer_ablation)

In [94]:
import json
for combi in exp_combis:
    experiment_type = combi[0]
    buffers = combi[1]
    
    src_date = "2023-06-17"     
    src_dir = f"../datasets/processed/post-review-tt-splits/{src_date}"
    tune = pd.read_csv(os.path.join(src_dir, "tune.csv"))
    tune = prepare_df(tune)


    # target
    target = 'agg_speed'

    # features
    time_cols = ['hour', 'dayofweek']

    road_cols = [
        'number_of_lanes', 'speed_limit_kph', 'elevation',
    ]

    landuse_cols = []
    if buffers=="all":
        buffer_sizes = [50, 100, 150, 200, 250, 500, 1000]
    else:
        buffer_sizes = buffers

    for size in buffer_sizes:
        landuse_cols.extend(
            [
                f'pix_residential_4x4_{size}',
                f'pix_institutional_4x4_{size}',
                f'pix_industrial_4x4_{size}',
                f'pix_business_4x4_{size}',
            ]
        )
    cat_cols = time_cols

    if experiment_type == "all_features":
        features = time_cols + road_cols + landuse_cols
        cat_cols_index = [
            index for (index, col) in enumerate(features) if col in cat_cols]

    elif experiment_type == "landuse_and_time":
        features = time_cols + landuse_cols
        cat_cols_index = [
            index for (index, col) in enumerate(features) if col in cat_cols]

    else:
        features = landuse_cols
        cat_cols_index = None


    X, y = tune[features], tune[target]

    # set up cv
    tscv = TimeSeriesSplit(n_splits=3)

    def objective(trial):
        """Objective function for optuna tuning. Uses LGBM sklearn API."""
        lgb_params = {
            'tree_learner': 'data',
            'seed':11,
            'verbose': -1,
    #         'boosting_type': trial.suggest_categorical(
    #             'boosting_type', ['goss', 'dart']),
            'boosting_type': 'goss',

            'objective': trial.suggest_categorical(
                'objective', ['regression', 'tweedie', 'regression_l1', 'mape']),

            'num_leaves': trial.suggest_int('num_leaves', 2, 512, log=True),
            'min_data_in_leaf': trial.suggest_int(
                'min_data_in_leaf', 100, 10000, log=True),

            'max_bin': trial.suggest_int(
                'max_bin', 100, 300),

            'n_estimators': trial.suggest_int("n_estimators", 100, 1000),
            'learning_rate' : trial.suggest_float("learning_rate", 0.001, 1, log=True),
        }
        if lgb_params['boosting_type']=='goss':
            top_rate =  trial.suggest_float("top_rate", 0, 1.0)
            other_rate = trial.suggest_float(
                "other_rate", 0, (1.0 - top_rate)) 

            # corrects the top_rate/other_rate values to only sum to 1
            if (top_rate + other_rate) > 1.0:
                lgb_params['other_rate'] = other_rate / (top_rate + other_rate)
                lgb_params['top_rate'] = top_rate / (top_rate + other_rate)


        model = lgb.LGBMRegressor(**lgb_params)
        scores = cross_val_score(
            estimator=model, X=X, y=y, scoring=rmse_scorer, cv=tscv, n_jobs=-1, 
            fit_params={'categorical_feature': cat_cols_index},
            error_score='raise')

        return np.mean(scores)

    study_name = f'{experiment_type}'
    if buffers=='all':
        buffers = buffers
    else:
        buffers = "-".join([str(buffers[0]), str(buffers[-1])])

    save_dir = f'../experiments/post-review/{buffers}/{src_date}'
    
    if os.path.exists(os.path.join(save_dir, f"{study_name}.db")):
        print(f"Done: {buffers}-{experiment_type}")
        continue
    else:
        print(f"Doing: {buffers}-{experiment_type}")
        
    os.makedirs(save_dir, exist_ok=True)    

    n_trials = 50
    study = optuna.create_study(study_name=study_name,
                                direction='minimize',
                                storage=f'sqlite:///{save_dir}/{study_name}.db',
                                load_if_exists=True)

    study.optimize(objective, n_trials=n_trials)     
    # change to 50; 
    # Hover to "Kernel" >> "Restart and Run All"
    
    params = study.best_params
    score = study.best_value

    print("Best params: ", params)
    print("Best score: ", score)

    fixed_params = {
        'tree_learner': 'data',
        'seed':11,
        'verbose': -1,
        'boosting_type': 'goss',
    }

    params.update(fixed_params)

    src_dir = f"../datasets/processed/post-review-tt-splits/{src_date}"
    train = pd.read_csv(os.path.join(src_dir, "train.csv"))
    train = prepare_df(train)

    test = pd.read_csv(os.path.join(src_dir, "test.csv"))
    test = prepare_df(test)

    # set up cv
    tscv = TimeSeriesSplit(n_splits=5)
    results = {}

    for i, (train_index, test_index) in enumerate(tscv.split(train)):
        print(f"Current: Fold {i}")
        fold_train = train.iloc[train_index]
        fold_test = train.iloc[test_index]

        X_train, y_train = fold_train[features], fold_train[target]
        X_test, y_test = fold_test[features], fold_test[target]    

        model = lgb.LGBMRegressor(**params)
        model.fit(X_train, y_train, categorical_feature=cat_cols_index)

        preds = model.predict(X_test)
        true = y_test.values

        # log per fold
        results[i] = {
            "y_true": list(true),
            "y_pred": list(preds),
        }

        for metric, func in metrics_dict.items():
            results[i][metric] = func(true, preds)

    tscv_results_dir = f"../datasets/results/post-review/tscv/{buffers}/{src_date}"
    os.makedirs(tscv_results_dir, exist_ok=True)

    fname = f"{experiment_type}.json"
    with open(os.path.join(tscv_results_dir, fname), 'w') as fp:
        json.dump(results, fp)


    X_train, y_train = train[features], train[target]
    X_test, y_test = test[features], test[target]    

    # retrain on full training set
    model = lgb.LGBMRegressor(**params)
    model.fit(X_train, y_train, categorical_feature=cat_cols_index)

    # evaluate on holdout set
    preds = model.predict(X_test)
    true = y_test.values

    # log final results
    final_results = {
        "y_true": list(true),
        "y_pred": list(preds),
    }

    for metric, func in metrics_dict.items():
        final_results[metric] = func(true, preds)

    final_results_dir = f"../datasets/results/post-review/final_results/{buffers}/{src_date}"
    os.makedirs(final_results_dir, exist_ok=True)

    fname = f"{experiment_type}.json"
    with open(os.path.join(final_results_dir, fname), 'w') as fp:
        json.dump(final_results, fp)

Done: all-all_features
Doing: 50-500-all_features


[I 2023-06-19 11:29:08,359] A new study created in RDB with name: all_features
/home/mdorosan/.local/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
[I 2023-06-19 11:29:30,571] Trial 0 finished with value: 7.488195922612989 and parameters: {'learning_rate': 0.005047090246423446, 'max_bin': 214, 'min_data_in_leaf': 3533, 'n_estimators': 607, 'num_leaves': 2, 'objective': 'regression_l1', 'other_rate': 0.492285906425415, 'top_rate': 0.31210312604228685}. Best is trial 0 with value: 7.488195922612989.
/home/mdorosan/.loca

/home/mdorosan/.local/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


[LightGBM] [Warning] min_data_in_leaf is set=1393, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1393


[I 2023-06-19 11:36:43,750] Trial 12 finished with value: 6.500151473356353 and parameters: {'learning_rate': 0.01664383903381375, 'max_bin': 227, 'min_data_in_leaf': 1108, 'n_estimators': 972, 'num_leaves': 83, 'objective': 'tweedie', 'other_rate': 0.026954203859002407, 'top_rate': 0.822504437867918}. Best is trial 4 with value: 6.416783999001338.
/home/mdorosan/.local/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
[I 2023-06-19 11:37:31,242] Trial 13 finished with value: 6.356553632984706 and parameters: {'learning_

...

[LightGBM] [Warning] min_data_in_leaf is set=106, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=106
[LightGBM] [Warning] min_data_in_leaf is set=172, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=172
[LightGBM] [Warning] min_data_in_leaf is set=346, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=346
[LightGBM] [Warning] min_data_in_leaf is set=237, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=237
[LightGBM] [Warning] min_data_in_leaf is set=237, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=237
[LightGBM] [Warning] min_data_in_leaf is set=101, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=101
[LightGBM] [Warning] min_data_in_leaf is set=101, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=101
[LightGBM] [Warning] min_data_in_leaf is set=277, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=277
[LightGB

[I 2023-06-19 17:33:29,450] Trial 37 finished with value: 7.577890736636608 and parameters: {'learning_rate': 0.0013664686320009145, 'max_bin': 166, 'min_data_in_leaf': 190, 'n_estimators': 873, 'num_leaves': 199, 'objective': 'regression_l1', 'other_rate': 0.018333142190758555, 'top_rate': 0.9587334964860252}. Best is trial 25 with value: 7.306690019079572.
/home/mdorosan/.local/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
[I 2023-06-19 17:34:33,905] Trial 38 finished with value: 16.41610148039896 and parameters: {

[LightGBM] [Warning] min_data_in_leaf is set=282, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=282
[LightGBM] [Warning] min_data_in_leaf is set=189, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=189
[LightGBM] [Warning] min_data_in_leaf is set=487, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=487
[LightGBM] [Warning] min_data_in_leaf is set=374, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=374
[LightGBM] [Warning] min_data_in_leaf is set=185, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=185
[LightGBM] [Warning] min_data_in_leaf is set=150, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=150
[LightGBM] [Warning] min_data_in_leaf is set=229, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=229
[LightGBM] [Warning] min_data_in_leaf is set=1159, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1159
[Light

[I 2023-06-19 17:40:48,560] Trial 45 finished with value: 7.2777462317203785 and parameters: {'learning_rate': 0.007941885199894751, 'max_bin': 145, 'min_data_in_leaf': 125, 'n_estimators': 746, 'num_leaves': 148, 'objective': 'regression_l1', 'other_rate': 0.05736438625375666, 'top_rate': 0.6879697138656098}. Best is trial 45 with value: 7.2777462317203785.
/home/mdorosan/.local/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
[I 2023-06-19 17:41:31,449] Trial 46 finished with value: 7.409158121412904 and parameters: {

Best params:  {'learning_rate': 0.007941885199894751, 'max_bin': 145, 'min_data_in_leaf': 125, 'n_estimators': 746, 'num_leaves': 148, 'objective': 'regression_l1', 'other_rate': 0.05736438625375666, 'top_rate': 0.6879697138656098}
Best score:  7.2777462317203785
[LightGBM] [Warning] min_data_in_leaf is set=101, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=101
[LightGBM] [Warning] min_data_in_leaf is set=101, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=101
[LightGBM] [Warning] min_data_in_leaf is set=277, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=277
[LightGBM] [Warning] min_data_in_leaf is set=842, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=842
[LightGBM] [Warning] min_data_in_leaf is set=114, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=114
[LightGBM] [Warning] min_data_in_leaf is set=196, min_child_samples=20 will be ignored. Current value: min_data_in_l

### Metadata reference

In [106]:
# fold index as key
results.keys()

dict_keys([0, 1, 2, 3, 4])

In [107]:
# keys per fold
results[0].keys()

dict_keys(['y_true', 'y_pred', 'r2', 'mean_absolute_error', 'mean_absolute_percentage_error', 'rmse', 'mean_squared_error', 'max_error'])

In [108]:
# keys per feature set and buffer combi
final_results.keys()

dict_keys(['y_true', 'y_pred', 'r2', 'mean_absolute_error', 'mean_absolute_percentage_error', 'rmse', 'mean_squared_error', 'max_error'])

## Verify results

In [104]:
# SANITY CHECK
from itertools import product

buffer_ablation = [
    'all',
    [50, 100, 150, 200, 250, 500],
    [50, 100, 150, 200, 250],
    [50, 100, 150, 200],
    [50, 100, 150],
    [50, 100],
    [1000],
    [500],
    [250],
    [200],
    [150],
    [100],
    [50], 
]


etypes = [
    "all_features",
    "landuse_and_time", # queue this with the ablation
    "landuse_only",
]

exp_combis = product(etypes, buffer_ablation)

for combi in exp_combis:
    experiment_type = combi[0]
    buffers = combi[1]
    
    study_name = f'{experiment_type}'
    
    if buffers=='all':
        buffers = buffers
    else:
        buffers = "-".join([str(buffers[0]), str(buffers[-1])])

    optuna_dir = f'../experiments/post-review/{buffers}/{src_date}'

    
    if not os.path.exists(os.path.join(optuna_dir, f"{study_name}.db")):
        print(f"NO OPTUNA STUDY: {buffers}-{experiment_type}")
    
    
    tscv_results_dir = f"../datasets/results/post-review/tscv/{buffers}/{src_date}"
    fname = f"{experiment_type}.json"
    if not os.path.exists(os.path.join(tscv_results_dir, fname)):
        print(f"NO TSCV: {buffers}-{experiment_type}")
        
    final_results_dir = f"../datasets/results/post-review/final_results/{buffers}/{src_date}"
    fname = f"{experiment_type}.json"
    if not os.path.exists(os.path.join(final_results_dir, fname)):
        print(f"NO FINAL RESULTS: {buffers}-{experiment_type}")
    

# End